In [1]:
import re
import string
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
import torch.cuda
from torch.utils.data import TensorDataset,DataLoader
import torch.nn as nn 
import _pickle as pickle
from string import punctuation

In [2]:
# đọc dữ liệu và add nó vào một mảng 
reviews_train = []
for line in open('document.txt', 'r'):
        reviews_train.append(line)
print(reviews_train[:30])

['3 mọi thứ luôn cần có thời_gian thay_đổi . \n', '3 ủa chứ cứ phải đứng trên bục cầm giơ giơ cái huy_chương mới là vô_địch ah mà mấy bác gắt quá \n', '3 cái gì cũng cần có thời_gian nhưng_mà vài người không đợi được thôi 🤣🤣🤣 \n', '1 công_nghệ phát_triển thể_này , vn có ô_tô made in vn rồi kb sau_này có máy_bay vietjet made in vn :)) \n', "1 nhin ngau_`_do ' . co_' len ... \n", '3 nếu vn chỉ chọn ra 1 ng lánh_đạo v tốt quá . nếu đất_nước có chuyện j hay ko pt đc thì đưa ông ra chịu toàn_bộ trách_nhiệm và đuổi đi lm thứ_dân là song \n', '3 yumi_phương nè xem đi chế ơi \n', '3 anh hiểu vì_sao có những lúc im_lặng ko_phải là vàng mà là chất kích nổ chưa ạ =)) minh nguyen \n', '2 bài báo thiếu vụ khủng khoảng marketing khi pepsi xù giải_thưởng với dân philip nữa xd \n', '2 chắc_ăn của qua 1 mớ nên cái trang này suốt ngày bênh_vực ! \n', '1 anh ngo đẹp phết a ah \n', '3 lên đi .. rồi trảm thằng tổng bí_thư \n', '1 mãi cfbiz mới có một bài hay và có chiều sâu \n', '3 book grab từ nội_bài về 

In [3]:
# tách data và labels
data = []
labels = []
for i in reviews_train:
        a = re.match(r"[\d +]", i)
        if a:
                if (i[:1] == "1" or i[:1] == "2"):
                        labels.append(i[:1])
                        data.append(i[2:])

In [4]:
labels = labels[:6500]
data = data[:6500]
#  loại bỏ hết ký tự trong phần data chỉ lấy chữ và dấu cách trống
reviews = []
for i in data:
        i = i.lower()
        reviews.append(re.sub(r'[^\w\s]','',i))

In [5]:
# Xây dựng bộ từ điển ánh xạ từ thành các số nguyên
all_text = ' '.join(reviews) # join các dòng
words = all_text.split() # tạo 1 list của các
counts = Counter(words) # đếm từ 
vocab = sorted(counts, key=counts.get, reverse=True) # xây dựng bộ từ điển 
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
print(vocab_to_int)

{'có': 1, 'là': 2, 'và': 3, 'của': 4, 'cho': 5, 'này': 6, 'các': 7, 'mình': 8, 'mà': 9, 'được': 10, 'thì': 11, 'đi': 12, 'bạn': 13, 'không': 14, 'đã': 15, 'hay': 16, 'với': 17, 'để': 18, 'quá': 19, 'trong': 20, 'nhiều': 21, 'những': 22, 'cũng': 23, 'người': 24, 'ko': 25, 'kotex': 26, 'cái': 27, 'rồi': 28, '1': 29, 'rất': 30, 'một': 31, 'làm': 32, 'mới': 33, 'lại': 34, 'đến': 35, 'nhé': 36, 'anh': 37, 'gì': 38, 'hơn': 39, 'còn': 40, 'xem': 41, 'như': 42, 'ngày': 43, 'về': 44, 'sẽ': 45, 'vnpt': 46, 'phim': 47, 'luôn': 48, 'từ': 49, '3': 50, 'em': 51, 'thấy': 52, 'vào': 53, 'nào': 54, 'bị': 55, 'cả': 56, 'chỉ': 57, 'tại': 58, 'nhất': 59, 'ở': 60, 'nó': 61, 'đẹp': 62, 'nữa': 63, 'đây': 64, 'ra': 65, 'con': 66, 'biết': 67, '2': 68, 'nhưng': 69, 'thích': 70, 'mua': 71, 'ai': 72, 'tôi': 73, 'vẫn': 74, 'phải': 75, 'lắm': 76, 'nên': 77, 'đỏ': 78, 'khi': 79, 'k': 80, 'nhà': 81, 'nestle': 82, 'vì': 83, 'cùng': 84, 'trên': 85, 'lên': 86, 'hàng': 87, 'đang': 88, 'rồng': 89, 'xe': 90, 'chương_trình'

In [6]:
# mảng dữ liệu gồm các từ đã được chuyển hóa thành số
reviews_ints = []
for r in reviews:
        reviews_ints.append([vocab_to_int[word] for word in r.split()])

In [7]:
# quy định lại labels cho tập dữ liệu
encoded_labels = np.array([1 if label=='1' else 0 for label in labels])

In [8]:
# can bang do dai so tu trong cac cau
# ta có seq_length là mức nếu một câu có chứa số từ ít hơn seq_length thì ta đệm số "0" vào, 
# còn những câu có độ dài dài hơn seq_length thì ta lấy [:seq_length] từ đầu tiên
def pad_features(reviews_ints, seq_length):
        features=np.zeros((len(reviews_ints), seq_length), dtype=int)
        for i, row in enumerate(reviews_ints):
                features[i, -len(row):] = np.array(row)[:seq_length]
        return features
seq_length = 100
features = pad_features(reviews_ints, seq_length=seq_length)


In [9]:
# chia tập dữ liệu thành 3 phần
# train : 80% sử dụng cho việc học dữ liệu
# validation: 10% sử dụng để điều chỉnh tham số 
# test: 10% sử dụng để đánh giá kết quả chương trình
train_x, remaining_x, train_y, remaining_y = train_test_split(features, encoded_labels, test_size=0.2)
test_x, val_x, test_y, val_y = train_test_split(remaining_x, remaining_y, test_size=0.5)
print(len(train_x))
print(len(train_y))

5200
5200


In [10]:
# DataLoaders và Batching
# tạo Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [11]:
# Sentiment Network trình bày ở cuối báo cáo

class SentimentRNN(nn.Module):
        # Sử dụng mô hình RNN

        def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

                super(SentimentRNN, self).__init__()

                self.output_size = output_size
                self.n_layers = n_layers
                self.hidden_dim = hidden_dim
                # xác định các lớp
                # embeding và LSTM các lớp
                self.embedding = nn.Embedding(vocab_size, embedding_dim)
                self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                                dropout=drop_prob, batch_first=True)
                self.dropout = nn.Dropout(0.3)
                self.fc = nn.Linear(hidden_dim, output_size)
                self.sig = nn.Sigmoid()

        def forward(self, x, hidden):
                batch_size = x.size(0)
                x = x.long()
                embeds = self.embedding(x)
                lstm_out, hidden = self.lstm(embeds, hidden)
                lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
                out = self.dropout(lstm_out)
                out = self.fc(out)
                sig_out = self.sig(out)
                sig_out = sig_out.view(batch_size, -1)
                sig_out = sig_out[:, -1] 
                return sig_out, hidden
    
    
        def init_hidden(self, batch_size):
        
                weight = next(self.parameters()).data
        
                hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
                return hidden


In [12]:
vocab_size = len(vocab_to_int) + 1 # kích thước từ vựng
output_size = 1 # kích thước đầu ra
embedding_dim = 400 # số cột trong bảng embeding
hidden_dim = 256 # số lượng đơn vị trong các hidden. Các giá trị phổ biển thường là 128, 256, 512, .... 
n_layers = 2 # số lớp của LMST , thông thường là từ 1-3
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

In [13]:
# training
# learning rate : tỉ lệ học tập cho tối ưu hóa
lr=0.001 
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
print(optimizer)


# Số lần lặp lại thông qua tập dữ liệu đào tạo
# để xác định được con số cụ thể thì ta phải quan sát xem số lần mà tại đó sự mất mát ngưng hoặc giảm
# cái này em chưa quan sát đang lấy một con số ngẫu nhiên để tăng độ chính xác
epochs = 4

counter = 0
print_every = 100
clip=5 
net.train()

for e in range(epochs):
        # khởi tạo trạng thái hidden
        h = net.init_hidden(batch_size)
        for inputs, labels in train_loader:

                counter += 1
                h = tuple([each.data for each in h])
                net.zero_grad()

                # lấy đấu ra từ mô hình
                output, h = net(inputs, h)
                
                loss = criterion(output.squeeze(), labels.float())
                loss.backward()
                # clip_grad_norm_ giúp ngăn chặn sự cố trong việc RNNs / LSTMs.
                nn.utils.clip_grad_norm_(net.parameters(), clip)
                optimizer.step()
                if (counter % print_every == 0):
                        val_losses = []
                        val_h = net.init_hidden(batch_size)
                        net.eval()
                        for inputs, labels in valid_loader:
                                val_h = tuple([each.data for each in val_h])
                                output, val_h = net(inputs, val_h)
                                val_loss = criterion(output.squeeze(), labels.float())
                                val_losses.append(val_loss.item())
                        net.train()
                        print("Epoch: {}/{}...".format(e+1, epochs),
                                "Step: {}...".format(counter),
                                "Loss: {:.6f}...".format(loss.item()),
                                "Val Loss: {:.6f}".format(np.mean(val_losses)))




Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Epoch: 1/4... Step: 100... Loss: 0.393016... Val Loss: 0.380703
Epoch: 2/4... Step: 200... Loss: 0.268067... Val Loss: 0.406350
Epoch: 3/4... Step: 300... Loss: 0.084360... Val Loss: 0.406032
Epoch: 4/4... Step: 400... Loss: 0.088899... Val Loss: 0.593295


In [14]:
# tính ti=oán độ chính xác

test_losses = []
num_correct = 0

# taọ các biến mới cho hidden
h = net.init_hidden(batch_size)
net.eval()

for inputs, labels in test_loader:
    h = tuple([each.data for each in h])   
    #  đầu ra ~ kết quả dự đoán
    output, h = net(inputs, h)
    
    # tính độ mất dl
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # chuyển đổi lớp đầu ra thành 0 hoặc 1 cũng ta sẽ làm tròn đến số nguyên gần nhất
    pred = torch.round(output.squeeze())  
    
    # so sánh dự đoán với nhãn thật của nó
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)



print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))



Test loss: 0.668
Test accuracy: 0.822


In [19]:
test_review_neg = "xấu mù "
test_review_pos = "hay quá "

def tokenize_review(test_review):
    test_review = test_review.lower() 
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])
    return test_ints

def predict(net, test_review, sequence_length):
    net.eval()
    test_ints = tokenize_review(test_review)
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    h = net.init_hidden(batch_size)
    output, h = net(feature_tensor, h)
    pred = torch.round(output.squeeze()) 
    print(pred.item())
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))    
    if(pred.item()==1):
        print("Positive review ")
    else:
        print("Negative review")

seq_length=30
predict(net, test_review_neg, seq_length)
predict(net, test_review_pos, seq_length)


0.0
Prediction value, pre-rounding: 0.026891
Negative review
1.0
Prediction value, pre-rounding: 0.999672
Positive review 
